<a href="https://colab.research.google.com/github/stevenrdz/IR-SOES/blob/master/VSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from time import time
start_nb = time()

start = time()
from nltk import SnowballStemmer
from nltk.corpus import stopwords
from collections import Counter
from nltk import word_tokenize
import nltk
import string
import numpy as np
import pandas as pd
import re
import math
import os

es_stops = set(stopwords.words('spanish'))
print('Importación de Librerias. %.2f segundos' % (time() - start))

Importación de Librerias. 0.00 segundos


In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
start = time()
dataset = pd.read_csv('//content/gdrive/My Drive/df.csv',
                      sep=';', encoding='utf-8-sig')


print('Dataset generado. %.2f segundos' % (time() - start))
dataset.head()

Dataset generado. 0.05 segundos


,Id,Title,Body
0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...
1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...
2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...
3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...
4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...


In [ ]:
def preprocess(doc):
    code = re.compile('[¿?""(),:'']') # Eliminación de signos de puntuacion.
    doc = re.sub(code, '', doc)
    punt = re.compile('[./@_]') # Eliminación de signos de puntuacion.
    doc = re.sub(punt, ' ', doc)
    code = re.compile('<pre><code>[\s\S]*</code></pre>')
    doc = re.sub(code, '', doc)
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [ ]:
start = time()
dataset['Processed_Title'] = dataset.apply(
        lambda dataset: preprocess(dataset['Title']), axis=1)
dataset['Processed_Body'] = dataset.apply(
        lambda dataset: preprocess(dataset['Body']), axis=1)
print('Preprocesado completo. %.2f segundos' % (time() - start))
dataset.iloc[:,[3,4]]

Preprocesado completo. 1.24 segundos


,Processed_Title,Processed_Body
0,"[debugear, código, django, utilizando, docker]","[cuento, realizando, proyecto, django, utilizo..."
1,"[file, conf, muestra, error, usando, superviso...","[configurar, file, conf, ruta, sudo, nano, etc..."
2,"[ejecutar, navegador, pantalla, completa, inic...","[realizando, proyecto, domotico, siguientes, c..."
3,"[pagina, web, blanco]","[buen, dia, pagina, web, sale, blanco, alguien..."
4,"[obtengo, error, i, choreographer, skipped, fr...","[trabajando, android, studio, intentando, hace..."
...,...,...
1079,"[conexion, remota, sql, server, linux, mint]","[instalado, sql, server, linux, mint, base, da..."
1080,"[crear, base, datos, centos, funciona]","[utilizando, centos, seguí, instrucciones, sit..."
1081,"[hacer, respaldo, base, datos, terminal]","[base, datos, ms, sql, server, express, ubuntu..."
1082,"[aplicación, windows, linux, wine]","[aplicación, desarrollada, vb, net, deve, xpre..."


In [ ]:
p_title = dataset['Processed_Title'].to_list()
p_body = dataset['Processed_Body'].to_list()

In [ ]:
N = len(dataset) # N = Cantidad de documentos
alpha = 0.3
DF = {}

for i in range(N):
  for w in p_title[i]:
      try:
        DF[w].add(i)
      except:
        DF[w] = {i}
  for w in p_body[i]:
      try:
        DF[w].add(i)
      except:
        DF[w] = {i}

for i in DF:
        DF[i] = len(DF[i])

total_vocab_size = len(DF)
total_vocab = [x for x in DF]

{'debugear': 1, 'código': 146, 'django': 8, 'utilizando': 58, 'docker': 16, 'cuento': 6, 'realizando': 21, 'proyecto': 76, 'utilizo': 37, 'pycharm': 1, 'windows': 116, 'levantar': 8, 'servidor': 191, 'web': 82, 'base': 67, 'datos': 112, 'instalado': 94, 'debian': 70, 'linux': 481, 'traves': 10, 'maquina': 39, 'virtual': 41, 'instale': 35, 'problemas': 89, 'compatibilidad': 4, 'problema': 311, 'trabajar': 17, 'manera': 99, 'encuentro': 71, 'forma': 128, 'poder': 88, 'poner': 42, 'puntos': 7, 'interrupcion': 1, 'demás': 14, 'buscando': 25, 'lados': 2, 'encontre': 12, 'ninguna': 31, 'solución': 45, 'tal': 48, 'vez': 97, 'alguno': 15, 'pudo': 10, 'resolver': 25, 'dudas': 14, 'dejo': 26, 'archivo': 261, 'yml': 4, 'version': 52, 'services': 7, 'db': 29, 'image': 11, 'postgres': 8, 'container': 6, 'name': 51, 'environment': 6, 'user': 72, 'xxxxxxx': 1, 'password': 27, 'broker': 1, 'rabbitmq': 1, 'default': 26, 'volumes': 3, 'code': 35, 'build': 9, 'ports': 8, 'command': 37, 'manage': 5, 'py':

In [ ]:
def doc_freq(DF, word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [ ]:
    start = time()

    doc = 0
    tf_idf = {}

#comentar este bloque
    for i in range(N):
        tokens = p_body[i]
        counter = Counter(tokens + p_title[i])
        words_count = len(tokens + p_title[i])
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            idf = np.log((N + 1) / (df + 1))
            tf_idf[doc, token] = tf * idf
        doc += 1
#comentar este bloque hasta aqui

    doc = 0
    tf_idf_title = {}

    for i in range(N):
        tokens = p_title[i]
        counter = Counter(tokens)
        words_count = len(tokens)
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            # numerator is added 1 to avoid negative values
            idf = np.log((N + 1) / (df + 1))  
            tf_idf_title[doc, token] = tf * idf
        doc += 1

#comentar este bloque
    for i in tf_idf_title:
        tf_idf_title[i] *= alpha
#comentar este bloque hasta aqui
    for i in tf_idf_title:
        tf_idf[i] = tf_idf_title[i]

    # D = Matriz TF-IDF de terminos de documentos
    D = np.zeros((N, total_vocab_size)) 
    for i in tf_idf:
        try:
            ind = total_vocab.index(i[1])
            D[i[0]][ind] = tf_idf[i]
        except:
            pass

    print('Matriz TF-IDF de terminos de documentos generada. %.2f segundos' % (time() - start))
    print(D[0])
    print(D[1])

Matriz TF-IDF de terminos de documentos generada. 3.19 segundos
[0.37777129 0.11993416 0.28752664 ... 0.         0.         0.        ]
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
def print_doc(dataset, id):
    with pd.option_context('display.max_colwidth', 500):
        print(dataset.iloc[[id], [0,1]])

In [ ]:
def gen_vector(tokens):
        Q = np.zeros((len(total_vocab)))
        counter = Counter(tokens)
        words_count = len(tokens)
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            idf = math.log((N + 1) / (df + 1))
            try:
                ind = total_vocab.index(token)
                Q[ind] = tf * idf
            except:
                pass
        return Q

In [ ]:
def cosine_similarity(k, query):
        tokens = preprocess(query)
        print("\nQuery:", query)
        d_cosines = []
        query_vector = gen_vector(tokens)
        for d in D:
            d_cosines.append(cosine_sim(query_vector, d))
        index_sort = np.array(d_cosines).argsort()[-k:][::-1]
        value_array = np.array(d_cosines)
        value_sort = np.sort(value_array)[-k:][::-1]
        res = []
        for i in range(k):
            res.append((index_sort[i], value_sort[i]))
        return res

In [ ]:
start = time()
results = cosine_similarity(10, "Error al instalar MySQL 8 en Ubuntu 16.04")

print("\n")
for i in range(len(results)):
  print_doc(dataset, results[i][0])
  print("sim: ",results[i]) 
  print("-------------------------------------------------------")

print('Tiempo de ejecución: %.2f segundos' % (time() - start))



Query: Error al instalar MySQL 8 en Ubuntu 16.04


         Id                                      Title
421  287702  Error al instalar MySQL 8 en Ubuntu 16.04
sim:  (421, 0.6010471573580218)
-------------------------------------------------------
         Id                                                           Title
976  256711  Access denied for user 'root'@'localhost' (using password: NO)
sim:  (976, 0.3149742869465327)
-------------------------------------------------------
         Id                                                      Title
690  179534  Ubuntu 16.02 lts error al instalar e desinstalar paquetes
sim:  (690, 0.2855587396577637)
-------------------------------------------------------
         Id                             Title
420  170539  MYSQL no permite entrar al motor
sim:  (420, 0.2828070018532938)
-------------------------------------------------------
        Id                                                                                         T